In [1]:
import numpy as np
import pandas as pd 
import ipaddress
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)

In [2]:
creditcard_data = pd.read_csv(r"C:\Users\ephre\Documents\Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions\Data\Cleaned_Data\Creditcard_cleaned_Data.csv")
fraud_data = pd.read_csv(r"C:\Users\ephre\Documents\Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions\Data\Cleaned_Data\fraud_cleaned_Data.csv")
ip_to_country = pd.read_csv(r"C:\Users\ephre\Documents\Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions\Data\Cleaned_Data\ip_to_country_cleaned_Data.csv")

In [3]:
creditcard_data.sample(7)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
253700,156837.0,1.898599,-1.905178,-0.756815,-1.341302,-1.559029,-0.522743,-0.973251,-0.243121,-1.284909,1.415279,-0.453129,0.015823,1.716092,-0.743953,-0.123524,-0.612796,0.537808,-0.280153,-0.481757,0.031227,0.006000,0.161439,0.168764,1.110072,-0.351949,-0.253221,-0.001152,0.000554,200.00,0
186554,127444.0,1.967574,-0.477626,-0.431299,-0.045464,-0.524693,-0.029779,-0.728712,0.220480,1.196918,-0.024030,0.817635,0.693870,-0.919438,0.318670,0.172076,0.413376,-0.655246,0.274133,0.259546,-0.249754,-0.154799,-0.396729,0.438703,0.651200,-0.561314,-0.330384,-0.001183,-0.042477,0.77,0
73037,55083.0,-0.711434,1.216109,2.063610,1.382705,0.539143,0.146861,0.917074,0.052001,-0.985871,-0.280234,-1.428507,0.195022,0.757942,-0.469188,-1.404756,0.580762,-0.760607,-0.544445,-1.057063,-0.093277,-0.227046,-0.615208,-0.192568,-0.136094,0.203483,-0.310754,0.062028,0.098971,18.96,0
235772,148897.0,2.238004,-0.657235,-2.285109,-1.200306,0.272415,-0.810780,0.003822,-0.292218,-1.033955,1.102561,0.556482,-0.599541,-0.816547,0.671931,-0.543617,0.902028,-0.055637,-0.932759,1.126217,-0.025912,0.310792,0.754192,-0.052358,0.308716,0.372000,-0.013481,-0.091287,-0.084738,27.23,0
9885,14688.0,1.137099,0.521867,0.748055,2.591003,0.283253,0.669620,-0.121319,0.028962,0.591150,0.279457,1.240598,-1.227980,2.887104,1.231681,-2.165291,0.605580,-0.123214,0.206591,-0.200301,-0.055466,-0.402317,-0.812161,-0.063465,-0.550494,0.493545,-0.171972,-0.019372,0.006843,24.39,0
99198,67182.0,1.170842,0.171396,0.124102,0.490702,-0.302690,-1.055727,0.330816,-0.171242,-0.392812,0.114036,1.594286,0.789905,-0.558387,0.790810,0.100072,0.189132,-0.397864,-0.389517,0.315101,-0.071477,-0.306911,-1.044239,0.140220,0.511578,0.201996,0.107050,-0.075881,0.005257,34.95,0
159767,113313.0,2.005795,-0.108168,-2.188930,0.024050,0.818024,0.055739,-0.015527,0.012847,0.822607,-0.687066,0.234709,0.612719,0.001093,-1.324970,-0.707627,0.518172,0.414809,0.897535,0.821579,-0.047471,-0.252837,-0.598558,0.049977,-0.465015,0.057751,-0.078776,-0.022432,-0.029970,40.00,0


In [4]:
fraud_data.sample(7)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour
26572,119480,2015-04-10 14:04:56,2015-04-29 13:57:04,19.0,CBFTMRHOXWWKC,SEO,IE,M,25,2.557948e+09,0,13,2,4,10,14,4,29,13
26040,72523,2015-06-30 02:09:13,2015-09-17 15:36:43,42.0,GSQGWOJBSNINV,Ads,Safari,F,35,2.047789e+09,0,15,3,6,30,2,9,17,15
17694,352097,2015-03-01 00:19:21,2015-04-18 01:56:10,12.0,XWQCFAENLBRZG,SEO,FireFox,F,39,6.990589e+08,0,1,5,3,1,0,4,18,1
107214,168804,2015-01-24 03:48:14,2015-02-24 17:20:01,87.0,NVCKXNUAAJJND,Direct,Safari,M,39,3.730058e+09,0,17,1,1,24,3,2,24,17
34105,258617,2015-08-05 21:10:43,2015-11-07 18:58:50,43.0,TUEYHUPKCWSQM,Ads,IE,M,33,1.357063e+09,0,18,5,8,5,21,11,7,18
21985,54438,2015-08-15 22:29:38,2015-09-18 17:49:22,15.0,RTXHATDNDYGVJ,Direct,Chrome,F,50,2.243968e+09,0,17,4,8,15,22,9,18,17
134039,262763,2015-05-19 08:53:03,2015-07-12 00:40:10,21.0,RQLGOWOZFPHSH,Ads,Chrome,M,26,2.399581e+09,0,0,6,5,19,8,7,12,0


In [5]:
ip_to_country.sample(7)

,lower_bound_ip_address,upper_bound_ip_address,country
90319,3.284126e+09,3284126207,Ukraine
136731,3.639648e+09,3639656447,United States
124511,3.437330e+09,3437330687,Canada
28502,1.728246e+09,1728246783,India
70387,3.231306e+09,3231306495,United States
128676,3.470213e+09,3470229503,United States
6535,1.026294e+09,1026310143,Japan


In [6]:
# Assuming fraud_data['ip_address'] is already numeric (integer representation)
ip_to_country['lower_bound_ip_address'] = ip_to_country['lower_bound_ip_address'].astype(float).astype(int)
ip_to_country['upper_bound_ip_address'] = ip_to_country['upper_bound_ip_address'].astype(float).astype(int)

# Now, define a function to map IP to a country
def map_country(ip, ip_df):
    result = ip_df[(ip_df['lower_bound_ip_address'] <= ip) & (ip_df['upper_bound_ip_address'] >= ip)]
    return result['country'].values[0] if len(result) > 0 else np.nan

# Apply the function to map the IP addresses to their respective countries
fraud_data['country'] = fraud_data['ip_address'].apply(lambda x: map_country(x, ip_to_country))


In [7]:
import ipaddress

# Convert the ip_address column in fraud_data to integers
fraud_data['ip_address'] = fraud_data['ip_address'].apply(lambda x: int(ipaddress.ip_address(x)) if isinstance(x, str) else x)

# Convert the lower and upper IP address bounds in the ip_to_country DataFrame to integers
ip_to_country['lower_bound_ip_address'] = ip_to_country['lower_bound_ip_address'].apply(lambda x: int(ipaddress.ip_address(x)) if isinstance(x, str) else x)
ip_to_country['upper_bound_ip_address'] = ip_to_country['upper_bound_ip_address'].apply(lambda x: int(ipaddress.ip_address(x)) if isinstance(x, str) else x)

# Now map the country based on IP address
fraud_data['country'] = fraud_data['ip_address'].apply(lambda x: map_country(x, ip_to_country))

In [8]:
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,2,5,2,24,22,4,18,2,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,1,0,6,7,20,6,8,1,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,18,3,1,1,18,1,1,18,United States
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0,13,0,4,28,21,5,4,13,NaN
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,18,2,7,21,7,9,9,18,United States


In [9]:
fraud_data['country'] = fraud_data['ip_address'].apply(lambda x: map_country(x, ip_to_country))

creditcard_data.rename(columns={'Class': 'Class_creditcard'}, inplace=True)
fraud_data.rename(columns={'class': 'class_fraud'}, inplace=True)

missing_cols_in_fraud = set(creditcard_data.columns) - set(fraud_data.columns)
missing_cols_in_credit = set(fraud_data.columns) - set(creditcard_data.columns)

for col in missing_cols_in_fraud:
    fraud_data[col] = np.nan
for col in missing_cols_in_credit:
    creditcard_data[col] = np.nan

combined_data = pd.concat([fraud_data, creditcard_data], ignore_index=True)

# Inspect the merged dataset
combined_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class_fraud,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country,V5,V26,V27,V7,V13,V22,V28,V20,V2,V10,Time,V18,V9,V11,V14,V17,V12,V15,V24,V3,V19,V25,V21,V8,Amount,V1,Class_creditcard,V6,V4,V16,V23
0,22058.0,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39.0,7.327584e+08,0.0,2.0,5.0,2.0,24.0,22.0,4.0,18.0,2.0,Japan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,333320.0,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53.0,3.503114e+08,0.0,1.0,0.0,6.0,7.0,20.0,6.0,8.0,1.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1359.0,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53.0,2.621474e+09,1.0,18.0,3.0,1.0,1.0,18.0,1.0,1.0,18.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,150084.0,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41.0,3.840542e+09,0.0,13.0,0.0,4.0,28.0,21.0,5.0,4.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,221365.0,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45.0,4.155831e+08,0.0,18.0,2.0,7.0,21.0,7.0,9.0,9.0,18.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
combined_data.sample(10)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class_fraud,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country,V5,V26,V27,V7,V13,V22,V28,V20,V2,V10,Time,V18,V9,V11,V14,V17,V12,V15,V24,V3,V19,V25,V21,V8,Amount,V1,Class_creditcard,V6,V4,V16,V23
146725,176200.0,2015-04-27 07:38:58,2015-08-01 22:13:07,42.0,VJGMDYYVJILLR,Ads,IE,M,48.0,3.408960e+09,0.0,22.0,5.0,4.0,27.0,7.0,8.0,1.0,22.0,Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.179726,0.124973,-0.039203,0.077558,-0.622157,-0.751325,0.124126,0.190752,-0.716561,-0.650652,167359.0,-0.064949,-0.422343,0.304363,0.496467,-0.860648,0.447629,-1.225428,-0.359684,0.888072,-0.044337,-0.001036,-0.085693,0.091474,1.79,-1.528078,0.0,-1.559010,-0.827376,0.346829,0.082863
367026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.158531,0.458201,-0.047119,-0.656488,-0.137927,-0.051707,-0.058779,-0.058593,-0.608859,0.077906,140586.0,-0.579652,0.578920,-0.065273,0.364011,-0.451284,0.334789,0.639031,0.739617,-2.397013,-0.199086,-0.155712,-0.003402,0.840794,44.19,1.955480,0.0,3.482241,-0.677805,-0.049377,0.208377
254204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.514667,-0.132986,-0.299664,0.709625,-0.904073,-0.659740,-0.215806,-0.137518,0.657339,-0.297872,68676.0,0.460397,-0.208120,1.038071,-0.323488,-0.698695,-0.016862,-0.081862,-0.077360,1.132757,-0.358614,-0.430061,-0.222636,-0.165432,7.99,-0.315892,0.0,-0.667911,-0.466063,0.911908,-0.044375
394545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.410392,0.667360,-0.059333,0.053602,0.396062,0.612812,-0.044712,-0.104053,0.012746,-0.325619,152311.0,0.425547,0.407703,1.089417,-1.122056,0.616316,1.166207,-1.270998,0.767247,-1.739377,0.226640,0.208610,0.119374,-0.121708,12.36,1.995369,0.0,-0.494278,0.514695,0.052274,-0.005041
156426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.357868,-0.035263,-0.151999,0.044549,1.563151,-1.216284,-0.633511,0.878432,2.005317,1.129043,5276.0,0.484665,1.988478,1.114641,0.624032,0.128686,-2.839646,0.478763,0.852728,-0.764737,-0.062902,0.174441,-0.744876,0.889408,1.98,-1.703949,0.0,3.236854,-1.240713,0.492881,-0.010165
191283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.279067,0.350473,0.076589,0.351762,0.268649,-0.304241,0.141735,0.288733,0.299542,-0.446800,40204.0,-0.638398,-0.282939,-0.187794,-0.164722,0.568659,0.498220,0.135780,0.952172,2.061807,1.185227,-0.371685,-0.103556,-0.056058,50.91,-0.689644,0.0,-0.871835,0.925724,-0.845691,0.176342
197218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.366717,-0.589045,-0.144668,0.434946,0.001138,-0.590454,0.149819,0.101342,-0.193907,-0.047602,42724.0,-2.262734,-1.650075,-0.118048,-0.083326,0.428867,-0.733089,0.247756,0.044691,1.470482,-0.006503,0.320716,-0.098356,-0.129442,49.00,-1.209215,0.0,-1.012227,-1.680513,0.780889,0.030190
175402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.102032,-0.335758,0.009166,-0.325517,-0.207557,-0.525992,0.027362,0.135540,-0.288359,-0.058142,33205.0,-0.058789,0.162376,-0.541108,0.210862,-0.508294,0.239655,-0.140390,1.014269,-1.258588,0.336231,0.861774,-0.131565,0.826939,94.00,1.097950,0.0,3.637654,0.334493,-0.090009,-0.202052
68739,78802.0,2015-01-05 03:47:42,2015-01-05 03:47:43,28.0,EYHPICOKTIBJO,Ads,Chrome,F,28.0,2.642968e+09,1.0,3.0,0.0,1.0,5.0,3.0,1.0,5.0,3.0,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Drop columns with more than 65% missing values
combined_data = combined_data.dropna(thresh=combined_data.shape[0] * 0.20, axis=1)


In [12]:
# Example of filling missing values
combined_data['country'].fillna('Unknown', inplace=True)  # For categorical columns

C:\Users\ephre\AppData\Local\Temp\ipykernel_33544\3989163506.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_data['country'].fillna('Unknown', inplace=True)  # For categorical columns


In [13]:
# Fill missing values in numerical columns with the mean
combined_data['age'] = combined_data['age'].fillna(combined_data['age'].mean())

# Fill missing values in categorical columns with the mode
combined_data['sex'] = combined_data['sex'].fillna(combined_data['sex'].mode()[0])
combined_data['browser'] = combined_data['browser'].fillna(combined_data['browser'].mode()[0])
combined_data['Amount'] = combined_data['Amount'].fillna(combined_data['Amount'].mode()[0])


In [14]:
numerical_columns = ["purchase_value", "age", "class_fraud", "Class_creditcard", "V3", "V27", "V14", "V18", "V10", "V21", "V8", "V23", "V1", "V5", "V19", "V22", "V17", "V16", "V9", "V12", "V2", "V26", "V15", "V7", "V24", "V13", "V25", "V20", "V11", "V28", "V4", "V6", "purchase_month", "purchase_day", "purchase_hour"]

# Apply mean filling to all numerical columns
for col in numerical_columns:
    combined_data[col] = combined_data[col].fillna(combined_data[col].mean())

# Check the resulting DataFrame
combined_data.head()


,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class_fraud,hour_of_day,day_of_week,signup_month,signup_day,signup_hour,purchase_month,purchase_day,purchase_hour,country,V5,V26,V27,V7,V13,V22,V28,V20,V2,V10,Time,V18,V9,V11,V14,V17,V12,V15,V24,V3,V19,V25,V21,V8,Amount,V1,Class_creditcard,V6,V4,V16,V23
0,22058.0,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39.0,7.327584e+08,0.0,2.0,5.0,2.0,24.0,22.0,4.0,18.0,2.0,Japan,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,NaN,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198
1,333320.0,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53.0,3.503114e+08,0.0,1.0,0.0,6.0,7.0,20.0,6.0,8.0,1.0,United States,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,NaN,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198
2,1359.0,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53.0,2.621474e+09,1.0,18.0,3.0,1.0,1.0,18.0,1.0,1.0,18.0,United States,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,NaN,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198
3,150084.0,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41.0,3.840542e+09,0.0,13.0,0.0,4.0,28.0,21.0,5.0,4.0,13.0,Unknown,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,NaN,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198
4,221365.0,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45.0,4.155831e+08,0.0,18.0,2.0,7.0,21.0,7.0,9.0,9.0,18.0,United States,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,NaN,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198


In [15]:
# List of columns you want to drop
columns_to_drop = ['user_id', 'signup_time', 'purchase_time', 'device_id', 'source', 'hour_of_day', 'day_of_week', 'signup_month', 'signup_day', 'signup_hour', 'Time', 'ip_address']

# Drop the columns from the DataFrame
combined_data = combined_data.drop(columns=columns_to_drop)

# Check the resulting DataFrame
combined_data.head()


,purchase_value,browser,sex,age,class_fraud,purchase_month,purchase_day,purchase_hour,country,V5,V26,V27,V7,V13,V22,V28,V20,V2,V10,V18,V9,V11,V14,V17,V12,V15,V24,V3,V19,V25,V21,V8,Amount,V1,Class_creditcard,V6,V4,V16,V23
0,34.0,Chrome,M,39.0,0.0,4.0,18.0,2.0,Japan,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198
1,16.0,Chrome,F,53.0,0.0,6.0,8.0,1.0,United States,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198
2,15.0,Opera,M,53.0,1.0,1.0,1.0,18.0,United States,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198
3,44.0,Safari,M,41.0,0.0,5.0,4.0,13.0,Unknown,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198
4,39.0,Safari,M,45.0,0.0,9.0,9.0,18.0,United States,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198


In [16]:
# combined_data = combined_data.dropna()

In [17]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434838 entries, 0 to 434837
Data columns (total 39 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   purchase_value    434838 non-null  float64
 1   browser           434838 non-null  object 
 2   sex               434838 non-null  object 
 3   age               434838 non-null  float64
 4   class_fraud       434838 non-null  float64
 5   purchase_month    434838 non-null  float64
 6   purchase_day      434838 non-null  float64
 7   purchase_hour     434838 non-null  float64
 8   country           434838 non-null  object 
 9   V5                434838 non-null  float64
 10  V26               434838 non-null  float64
 11  V27               434838 non-null  float64
 12  V7                434838 non-null  float64
 13  V13               434838 non-null  float64
 14  V22               434838 non-null  float64
 15  V28               434838 non-null  float64
 16  V20               43

In [18]:
# Categorical columns to encode
categorical_columns = ['browser', 'sex', 'country']

# Apply One-Hot Encoding
combined_data = pd.get_dummies(combined_data, columns=categorical_columns, drop_first=True)

# Check the result
combined_data.head()


,purchase_value,age,class_fraud,purchase_month,purchase_day,purchase_hour,V5,V26,V27,V7,V13,V22,V28,V20,V2,V10,V18,V9,V11,V14,V17,V12,V15,V24,V3,V19,V25,V21,V8,Amount,V1,Class_creditcard,V6,V4,V16,V23,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_Albania,country_Algeria,country_Angola,country_Antigua and Barbuda,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Bahrain,country_Bangladesh,country_Barbados,country_Belarus,country_Belgium,country_Belize,country_Benin,country_Bermuda,country_Bhutan,country_Bolivia,country_Bonaire; Sint Eustatius; Saba,country_Bosnia and Herzegowina,country_Botswana,country_Brazil,country_British Indian Ocean Territory,country_Brunei Darussalam,country_Bulgaria,country_Burkina Faso,country_Burundi,country_Cambodia,country_Cameroon,country_Canada,country_Cape Verde,country_Cayman Islands,country_Chile,country_China,country_Colombia,country_Congo,country_Congo The Democratic Republic of The,country_Costa Rica,country_Cote D'ivoire,country_Croatia (LOCAL Name: Hrvatska),country_Cuba,country_Curacao,country_Cyprus,country_Czech Republic,country_Denmark,country_Djibouti,country_Dominica,country_Dominican Republic,country_Ecuador,country_Egypt,country_El Salvador,country_Estonia,country_Ethiopia,country_European Union,country_Faroe Islands,country_Fiji,country_Finland,country_France,country_Gabon,country_Gambia,country_Georgia,country_Germany,country_Ghana,country_Gibraltar,country_Greece,country_Guadeloupe,country_Guam,country_Guatemala,country_Haiti,country_Honduras,country_Hong Kong,country_Hungary,country_Iceland,country_India,country_Indonesia,country_Iran (ISLAMIC Republic Of),country_Iraq,country_Ireland,country_Israel,country_Italy,country_Jamaica,country_Japan,country_Jordan,country_Kazakhstan,country_Kenya,country_Korea Republic of,country_Kuwait,country_Kyrgyzstan,country_Lao People's Democratic Republic,country_Latvia,country_Lebanon,country_Lesotho,country_Libyan Arab Jamahiriya,country_Liechtenstein,country_Lithuania,country_Luxembourg,country_Macau,country_Macedonia,country_Madagascar,country_Malawi,country_Malaysia,country_Maldives,country_Malta,country_Mauritius,country_Mexico,country_Moldova Republic of,country_Monaco,country_Mongolia,country_Montenegro,country_Morocco,country_Mozambique,country_Myanmar,country_Namibia,country_Nauru,country_Nepal,country_Netherlands,country_New Caledonia,country_New Zealand,country_Nicaragua,country_Niger,country_Nigeria,country_Norway,country_Oman,country_Pakistan,country_Palestinian Territory Occupied,country_Panama,country_Papua New Guinea,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Qatar,country_Reunion,country_Romania,country_Russian Federation,country_Rwanda,country_Saint Kitts and Nevis,country_Saint Martin,country_San Marino,country_Saudi Arabia,country_Senegal,country_Serbia,country_Seychelles,country_Singapore,country_Slovakia (SLOVAK Republic),country_Slovenia,country_South Africa,country_South Sudan,country_Spain,country_Sri Lanka,country_Sudan,country_Sweden,country_Switzerland,country_Syrian Arab Republic,country_Taiwan; Republic of China (ROC),country_Tajikistan,country_Tanzania United Republic of,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_Turkmenistan,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,34.0,39.0,0.0,4.0,18.0,2.0,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198,False,False,False,Fals

In [19]:
boolean_columns = combined_data.select_dtypes(include="bool").columns

combined_data[boolean_columns] = combined_data[boolean_columns].astype(int)

In [20]:
combined_data.head()

,purchase_value,age,class_fraud,purchase_month,purchase_day,purchase_hour,V5,V26,V27,V7,V13,V22,V28,V20,V2,V10,V18,V9,V11,V14,V17,V12,V15,V24,V3,V19,V25,V21,V8,Amount,V1,Class_creditcard,V6,V4,V16,V23,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_Albania,country_Algeria,country_Angola,country_Antigua and Barbuda,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Bahrain,country_Bangladesh,country_Barbados,country_Belarus,country_Belgium,country_Belize,country_Benin,country_Bermuda,country_Bhutan,country_Bolivia,country_Bonaire; Sint Eustatius; Saba,country_Bosnia and Herzegowina,country_Botswana,country_Brazil,country_British Indian Ocean Territory,country_Brunei Darussalam,country_Bulgaria,country_Burkina Faso,country_Burundi,country_Cambodia,country_Cameroon,country_Canada,country_Cape Verde,country_Cayman Islands,country_Chile,country_China,country_Colombia,country_Congo,country_Congo The Democratic Republic of The,country_Costa Rica,country_Cote D'ivoire,country_Croatia (LOCAL Name: Hrvatska),country_Cuba,country_Curacao,country_Cyprus,country_Czech Republic,country_Denmark,country_Djibouti,country_Dominica,country_Dominican Republic,country_Ecuador,country_Egypt,country_El Salvador,country_Estonia,country_Ethiopia,country_European Union,country_Faroe Islands,country_Fiji,country_Finland,country_France,country_Gabon,country_Gambia,country_Georgia,country_Germany,country_Ghana,country_Gibraltar,country_Greece,country_Guadeloupe,country_Guam,country_Guatemala,country_Haiti,country_Honduras,country_Hong Kong,country_Hungary,country_Iceland,country_India,country_Indonesia,country_Iran (ISLAMIC Republic Of),country_Iraq,country_Ireland,country_Israel,country_Italy,country_Jamaica,country_Japan,country_Jordan,country_Kazakhstan,country_Kenya,country_Korea Republic of,country_Kuwait,country_Kyrgyzstan,country_Lao People's Democratic Republic,country_Latvia,country_Lebanon,country_Lesotho,country_Libyan Arab Jamahiriya,country_Liechtenstein,country_Lithuania,country_Luxembourg,country_Macau,country_Macedonia,country_Madagascar,country_Malawi,country_Malaysia,country_Maldives,country_Malta,country_Mauritius,country_Mexico,country_Moldova Republic of,country_Monaco,country_Mongolia,country_Montenegro,country_Morocco,country_Mozambique,country_Myanmar,country_Namibia,country_Nauru,country_Nepal,country_Netherlands,country_New Caledonia,country_New Zealand,country_Nicaragua,country_Niger,country_Nigeria,country_Norway,country_Oman,country_Pakistan,country_Palestinian Territory Occupied,country_Panama,country_Papua New Guinea,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Qatar,country_Reunion,country_Romania,country_Russian Federation,country_Rwanda,country_Saint Kitts and Nevis,country_Saint Martin,country_San Marino,country_Saudi Arabia,country_Senegal,country_Serbia,country_Seychelles,country_Singapore,country_Slovakia (SLOVAK Republic),country_Slovenia,country_South Africa,country_South Sudan,country_Spain,country_Sri Lanka,country_Sudan,country_Sweden,country_Switzerland,country_Syrian Arab Republic,country_Taiwan; Republic of China (ROC),country_Tajikistan,country_Tanzania United Republic of,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_Turkmenistan,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,34.0,39.0,0.0,4.0,18.0,2.0,0.001828,0.000149,0.001763,0.001801,0.000603,-0.000015,0.000547,0.000187,-0.004135,-0.001441,0.001515,-0.001596,0.000202,0.000252,0.00017,-0.000715,0.001043,0.000214,0.001613,-0.000264,-0.000232,-0.000371,-0.000854,1.0,0.005917,0.001667,-0.001139,-0.002966,0.001162,0.000198,0,0,0,0,1,0,0,0,0,0,0,

In [21]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd

# 1. Filter numeric columns
numeric_columns = combined_data.select_dtypes(include=['float64', 'int64']).columns

# 2. Initialize the StandardScaler
scaler = StandardScaler()

# 3. Fit and transform only the numeric columns
scaled_data = scaler.fit_transform(combined_data[numeric_columns])

# 4. Create a DataFrame with the scaled numeric columns
scaled_data_df = pd.DataFrame(scaled_data, columns=numeric_columns)

# 5. Replace the original numeric columns in `combined_data` with the scaled columns
combined_data[numeric_columns] = scaled_data_df

# Now all the numeric columns in combined_data are standardized


In [22]:
scaler_MinMax = MinMaxScaler()
combined_data[numerical_columns] = scaler.fit_transform(combined_data[numerical_columns])

In [23]:
combined_data.sample(7)

,purchase_value,age,class_fraud,purchase_month,purchase_day,purchase_hour,V5,V26,V27,V7,V13,V22,V28,V20,V2,V10,V18,V9,V11,V14,V17,V12,V15,V24,V3,V19,V25,V21,V8,Amount,V1,Class_creditcard,V6,V4,V16,V23,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_M,country_Albania,country_Algeria,country_Angola,country_Antigua and Barbuda,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Bahrain,country_Bangladesh,country_Barbados,country_Belarus,country_Belgium,country_Belize,country_Benin,country_Bermuda,country_Bhutan,country_Bolivia,country_Bonaire; Sint Eustatius; Saba,country_Bosnia and Herzegowina,country_Botswana,country_Brazil,country_British Indian Ocean Territory,country_Brunei Darussalam,country_Bulgaria,country_Burkina Faso,country_Burundi,country_Cambodia,country_Cameroon,country_Canada,country_Cape Verde,country_Cayman Islands,country_Chile,country_China,country_Colombia,country_Congo,country_Congo The Democratic Republic of The,country_Costa Rica,country_Cote D'ivoire,country_Croatia (LOCAL Name: Hrvatska),country_Cuba,country_Curacao,country_Cyprus,country_Czech Republic,country_Denmark,country_Djibouti,country_Dominica,country_Dominican Republic,country_Ecuador,country_Egypt,country_El Salvador,country_Estonia,country_Ethiopia,country_European Union,country_Faroe Islands,country_Fiji,country_Finland,country_France,country_Gabon,country_Gambia,country_Georgia,country_Germany,country_Ghana,country_Gibraltar,country_Greece,country_Guadeloupe,country_Guam,country_Guatemala,country_Haiti,country_Honduras,country_Hong Kong,country_Hungary,country_Iceland,country_India,country_Indonesia,country_Iran (ISLAMIC Republic Of),country_Iraq,country_Ireland,country_Israel,country_Italy,country_Jamaica,country_Japan,country_Jordan,country_Kazakhstan,country_Kenya,country_Korea Republic of,country_Kuwait,country_Kyrgyzstan,country_Lao People's Democratic Republic,country_Latvia,country_Lebanon,country_Lesotho,country_Libyan Arab Jamahiriya,country_Liechtenstein,country_Lithuania,country_Luxembourg,country_Macau,country_Macedonia,country_Madagascar,country_Malawi,country_Malaysia,country_Maldives,country_Malta,country_Mauritius,country_Mexico,country_Moldova Republic of,country_Monaco,country_Mongolia,country_Montenegro,country_Morocco,country_Mozambique,country_Myanmar,country_Namibia,country_Nauru,country_Nepal,country_Netherlands,country_New Caledonia,country_New Zealand,country_Nicaragua,country_Niger,country_Nigeria,country_Norway,country_Oman,country_Pakistan,country_Palestinian Territory Occupied,country_Panama,country_Papua New Guinea,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Qatar,country_Reunion,country_Romania,country_Russian Federation,country_Rwanda,country_Saint Kitts and Nevis,country_Saint Martin,country_San Marino,country_Saudi Arabia,country_Senegal,country_Serbia,country_Seychelles,country_Singapore,country_Slovakia (SLOVAK Republic),country_Slovenia,country_South Africa,country_South Sudan,country_Spain,country_Sri Lanka,country_Sudan,country_Sweden,country_Switzerland,country_Syrian Arab Republic,country_Taiwan; Republic of China (ROC),country_Tajikistan,country_Tanzania United Republic of,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_Turkmenistan,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Unknown,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
129560,2.042779e+00,5.628359e-01,-5.452672e-01,6.320715e-01,1.486069e+00,3.628074e-01,-5.692161e-19,8.835746e-18,4.600026e-18,-8.670515e-18,1.272106e-17,-2.321931e-17,5.909070e-18,-1.115401e-17,9.150625e-19,-3.008336e-18,-1.474997e-17,-1.005075e-17,-4.588374e-17,-2.757753e-18,-1.077146e-17,2.475714e-17,-4.005956e-17,-8.63741

In [24]:
combined_data.isnull().sum().sum()

np.int64(0)

In [25]:
file_path = r"C:/Users/ephre/Documents/Fraud-Detection-for-E-Commerce-and-Banking-Transactions---AI-Solutions/Data/Cleaned_Data/combined_data.csv"

combined_data.to_csv(file_path, index=False)